## How decision trees behave with different types of encoding
This example comes from [here](https://github.com/groverpr/Machine-Learning/blob/master/catboost/ohc_dtreeviz.ipynb)

### Imports

In [1]:
from sklearn.datasets import *
from sklearn import tree
from dtreeviz.trees import *
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder

### Data creation

Artificial dataset of size 1000 with one categorical variable. There are 100 different categories and target equals 1 for 10 of them and 0 for all others. **There is no noise in the dataset**

#### Code

In [2]:
binary_feature_with_high_cardinality = np.random.randint(0, 100, 1000)  # 100 cateogries for 1000 observations
X = pd.DataFrame({"x": binary_feature_with_high_cardinality})

# target_labels = np.random.binomial(1, 0.5, size=1000)  # 0 or 1 with 50-50 ratio

In [3]:
np.linspace(0,99,10)

array([ 0., 11., 22., 33., 44., 55., 66., 77., 88., 99.])

In [4]:
target = []
for i in binary_feature_with_high_cardinality:
    if i in np.linspace(0,99,10):
        target.append(1)
    else: 
        target.append(0)
        
target = np.array(target)

In [5]:
sum(target)

103

### Label encoding

Nothing to do here, categories are already label encoded.

In [6]:
classifier = tree.DecisionTreeClassifier(max_depth=5)  # limit depth of tree
classifier.fit(X, target)

viz = dtreeviz(classifier, 
               X, 
               target,
               target_name='target',
              feature_names= ['feature'],
              class_names = ["label-0", "label-1"]  # need class_names for classifier
              )  
              
viz.view()

/home/kv-dimitrov/.pyenv/versions/3.6.2/envs/generic/lib/python3.6/site-packages/matplotlib/font_manager.py:1241: UserWarning: findfont: Font family ['Arial'] not found. Falling back to DejaVu Sans.
  (prop.get_family(), self.defaultFamily[fontext]))


### One Hot Encoding

In [7]:
enc = OneHotEncoder(handle_unknown='ignore')
enc.fit(X)
X_ohc = enc.transform(X)
X_ohc = pd.DataFrame(X_ohc.toarray())

In [8]:
classifier = tree.DecisionTreeClassifier(max_depth=5)  # limit depth of tree
classifier.fit(X_ohc.values, target)

viz = dtreeviz(classifier, 
               X_ohc, 
               target,
               target_name='target',
              feature_names= [f'{c}_feature' for c in X_ohc.columns],
              class_names = ["label-0", "label-1"]  # need class_names for classifier
              )  
              
viz.view()

### K-Fold Target Encoding

In [9]:
from category_encoders.target_encoder import TargetEncoder
from sklearn.model_selection import KFold

In [10]:
encoder = TargetEncoder(cols="x")
encoder.fit(X, target)
X_encode = encoder.transform(X, target)

In [11]:
classifier = tree.DecisionTreeClassifier(max_depth=5)  # limit depth of tree
classifier.fit(X_encode, target)

viz = dtreeviz(classifier, 
               X_encode, 
               target,
               target_name='target',
              feature_names= X_encode.columns, 
              class_names = ["label-0", "label-1"]  # need class_names for classifier
              )  
              
viz.view()

### Catboost Encoding

In [12]:
def catboost_target_encoder(train, cols_encode, target):
    train_new = train.copy()
    for column in cols_encode:
        global_mean = train[target].mean()
        cumulative_sum = train.groupby(column)[target].cumsum() - train[target]
        cumulative_count = train.groupby(column).cumcount()
        train_new[column + "_cat_mean_target"] = cumulative_sum/cumulative_count
        train_new[column + "_cat_mean_target"].fillna(global_mean, inplace=True)
    return train_new

In [13]:
X_cat = catboost_target_encoder(pd.concat([X, pd.DataFrame({"y":target})], axis=1), "x", "y")
X_cat = pd.DataFrame({"x": X_cat["x_cat_mean_target"]})

In [14]:
classifier = tree.DecisionTreeClassifier(max_depth=5)  # limit depth of tree
classifier.fit(X_cat, target)

viz = dtreeviz(classifier, 
               X_cat, 
               target,
               target_name='target',
              feature_names= "x", 
              class_names = ["label-0", "label-1"]  # need class_names for classifier
              )  
              
viz.view()